## Foma de organizaçao de armazenamento

Técnicas utilizadas: 

- Partitioning
- Z-Order indexing - Usado em uma coluna incremental
- Liquid Clustering - Otimiza sem precisar reescrever os dados

| Técnica | Funcionamento Básico | Principais Vantagens | Desvantagens / Limitações |
| :--- | :--- | :--- | :--- |
| **Partitioning** | Divide os dados em pastas físicas no storage (ex: `/ano=2024/mes=01/`). | Extremamente eficiente para filtros básicos; reduz o escaneamento de pastas desnecessárias. | Sofre com "over-partitioning" (muitos arquivos pequenos) e não lida bem com alta cardinalidade. |
| **Z-Order Indexing** | Reorganiza os dados dentro dos arquivos Delta para manter valores relacionados próximos. | Excelente para colunas de alta cardinalidade (IDs, CPFs) e múltiplas colunas de busca. | Exige execução manual do comando `OPTIMIZE` e perde eficiência se os dados mudarem muito. |
| **Liquid Clustering** | Agrupa dados dinamicamente com base em chaves de cluster, sem depender de pastas fixas. | Flexível, evita problemas de skew, permite mudar as chaves sem reescrever a tabela toda e é autogerenciado. | Recurso mais recente (exige versões novas do DBR) e tem um custo computacional inicial para organizar. |

## Exemplo de usos (não rode, são apenas exemplos)

### Partitioning


In [0]:
%sql
-- SQL
CREATE TABLE eventos_particionados (
    id_evento INT,
    nome_evento STRING,
    data_evento DATE
)
USING DELTA
PARTITIONED BY (data_evento);

In [0]:
df.write.format("delta") \
  .partitionBy("data_evento") \
  .saveAsTable("eventos_particionados")

### Z-Order Indexing

In [0]:
%sql
OPTIMIZE eventos_particionados 
WHERE data_evento >= '2024-01-01'
ZORDER BY (id_evento);

In [0]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forName(spark, "eventos_particionados")
deltaTable.optimize().executeZOrderBy("id_evento")

### Liquid Clustering

In [0]:
%sql
-- Além de utilizado no CREATE TABLE, tbm é possivel utilizar em uma tabela já existente utilizando o ALTER TABLE
ALTER TABLE eventos_particionados
SET TBLPROPERTIES (
CREATE TABLE eventos_clustering (
    id_evento INT,
    nome_evento STRING,
    data_evento DATE
)
USING DELTA
CLUSTER BY (id_evento, data_evento); -- também é possivel deixar o databricks definir a melhor coluna utilizando "CLUSTER BY AUTO"

-- Para manter o clustering atualizado
OPTIMIZE eventos_clustering;

In [0]:
df.write.format("delta") \
  .clusterBy("id_evento", "data_evento") \
  .saveAsTable("eventos_clustering")